In [1]:
import os
import pandas as pd
import numpy as np
import librosa
from scipy.signal import lfilter

class FeatureExtractor:
    def __init__(self, n_mfcc=20):
        self.n_mfcc = n_mfcc

    def compute_mfccs(self, signal, sample_rate):
        """
        Tính toán Mel-frequency cepstral coefficients (MFCCs) của tín hiệu âm thanh.
        """
        mfccs = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=self.n_mfcc)
        return np.mean(mfccs.T, axis=0)

    def compute_zcr(self, signal):
        """
        Tính toán Zero Crossing Rate (ZCR) của tín hiệu âm thanh.
        """
        zcr = librosa.feature.zero_crossing_rate(signal)[0]
        zcr_mean = np.mean(zcr)
        zcr_variance = np.var(zcr)
        zcr_max = np.max(zcr)
        zcr_min = np.min(zcr)
        zcr_median = np.median(zcr)
        return zcr_mean, zcr_variance, zcr_max, zcr_min, zcr_median

    def extract_pitch(self, signal, sample_rate):
        # Tính toán pitch sử dụng hàm estimate_tuning của Librosa
        pitch, _ = librosa.core.piptrack(y=signal, sr=sample_rate)
        
        # Lấy giá trị pitch trung bình
        mean_pitch = pitch.mean()
        
        return mean_pitch
    
    def extract_rms_features(self, y, frame_length=512, hop_length=256):
        """Trích xuất RMS energy và tính các giá trị thống kê."""
        rms = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]
        rms_mean = np.mean(rms)
        rms_variance = np.var(rms)
        rms_max = np.max(rms)
        rms_min = np.min(rms)
        rms_median = np.median(rms)
        return rms_mean, rms_variance, rms_max, rms_min, rms_median
    
    import librosa

    def calculate_formants(file_path):
      # Load the audio file
      signal, sr = librosa.load(file_path)

      # Pre-emphasis
      pre_emphasized_signal = np.append(signal[0], signal[1:] - 0.97 * signal[:-1])

      # Compute the Linear Predictive Coding
      lpc_order = 2 + sr // 1000
      A = librosa.lpc(pre_emphasized_signal, order = lpc_order)
      lpc_coeffs = lfilter([0] + -1*A[1:], [1], signal)

      # Compute the roots of the LPC
      rts = np.roots(A)
      rts = [r for r in rts if np.imag(r) >= 0]

      # Compute the angles and sort them by frequency
      angz = np.arctan2(np.imag(rts), np.real(rts))
      frqs = sorted(angz * (sr / (2 * np.pi)))

      return frqs

    def feature_engineering_for_file(self, audio_file):
        """
        Trích xuất các đặc trưng MFCC, ZCR và Pitch từ một tệp âm thanh.
        """
        signal, sample_rate = librosa.load(audio_file, sr=None)
        if len(signal) >= 2048:
            mfcc_array = self.compute_mfccs(signal, sample_rate)
            zcr_mean, zcr_variance, zcr_max, zcr_min, zcr_median = self.compute_zcr(signal)
            pitch = self.extract_pitch(signal, sample_rate)
            rms_mean, rms_variance, rms_max, rms_min, rms_median = self.extract_rms_features(signal)
            # hnr = self.calculate_hnr(signal)
            return mfcc_array, zcr_mean, zcr_variance, zcr_max, zcr_min, zcr_median, pitch, rms_mean, rms_variance, rms_max, rms_min, rms_median
        return None

    def process_file(self, label, input_folder_path):
        """
        Xử lý tất cả các tệp WAV trong thư mục để trích xuất các đặc trưng.
        """
        # Tạo một DataFrame để lưu trữ đặc trưng MFCC
        mfcc_data = pd.DataFrame()

        for file_name in os.listdir(input_folder_path):
            if file_name.endswith('.wav'):
                file_path = os.path.join(input_folder_path, file_name)

                features = self.feature_engineering_for_file(file_path)
                if features is not None:
                    mfcc_array, zcr_mean, zcr_variance, zcr_max, zcr_min, zcr_median, pitch, rms_mean, rms_variance, rms_max, rms_min, rms_median = features
                    
                    mfcc_df = pd.DataFrame(mfcc_array.reshape(1, -1), columns=[f'mfcc_{i+1}' for i in range(len(mfcc_array))])

                    mfcc_df['label'] = label
                    mfcc_df['file'] = file_name
                    mfcc_df['zcr_mean'] = zcr_mean
                    mfcc_df['zcr_variance'] = zcr_variance
                    mfcc_df['zcr_max'] = zcr_max
                    mfcc_df['zcr_min'] = zcr_min
                    mfcc_df['zcr_median'] = zcr_median
                    mfcc_df['Pitch'] = pitch
                    mfcc_df['rms_mean'] = rms_mean
                    mfcc_df['rms_variance'] = rms_variance
                    mfcc_df['rms_max'] = rms_max
                    mfcc_df['rms_min'] = rms_min
                    mfcc_df['rms_median'] = rms_median
                    # mfcc_df['hnr'] = hnr
                    
                    mfcc_data = pd.concat([mfcc_data, mfcc_df], ignore_index=True)
            
        return mfcc_data

    def process_folder(self, input_folder, output_csv_path):
        """
        Xử lý tất cả các thư mục con trong thư mục đầu vào để trích xuất các đặc trưng và lưu chúng vào một tệp CSV.
        """
        # Tạo một DataFrame để lưu trữ dữ liệu MFCC
        mfcc_data = pd.DataFrame()

        # Duyệt qua tất cả các thư mục con trong thư mục đầu vào
        for root, dirs, files in os.walk(input_folder):
            for folder in dirs:
                # Xác định đường dẫn của thư mục con trong thư mục đầu vào
                subdirectory_input = os.path.join(root, folder)
                # Xử lý các tệp trong thư mục con và thêm dữ liệu MFCC vào DataFrame
                mfcc_df = self.process_file(folder, subdirectory_input)
                mfcc_data = pd.concat([mfcc_data, mfcc_df], ignore_index=True)

        # Lưu dữ liệu MFCC vào tệp CSV
        mfcc_data.to_csv(output_csv_path, index=False)

In [2]:
featureExtractor = FeatureExtractor(20)
featureExtractor.process_folder('D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/cleaned_data','feature.csv')

In [5]:
import librosa
import numpy as np

def calculate_hnr(file_path):
    # Tải tín hiệu âm thanh
    signal, sample_rate = librosa.load(file_path, sr=None)
    
    # Phân tách tín hiệu thành hài hòa và nhiễu
    harmonic, percussive = librosa.effects.hpss(signal)
    
    # Tính tổng năng lượng của hài hòa và nhiễu
    harmonic_energy = np.sum(harmonic ** 2)
    percussive_energy = np.sum(percussive ** 2)

    print(harmonic_energy, percussive_energy)
    
    # Tính HNR
    hnr = 10 * np.log10(harmonic_energy / (percussive_energy + 0.0000000001))  # Thêm 1e-6 để tránh chia cho 0
    
    return hnr

# Sử dụng chức năng
file_path = r'D:\data_analysis\speech_emotion_recognition\data\EnglishDataset\cleaned_data\Sad\03-01-04-01-01-01-09.wav'
hnr_value = calculate_hnr(file_path)
print(f"HNR: {hnr_value} dB")

0.017103037 0.026597798
HNR: -1.91772451070778 dB


In [3]:
import numpy as np
import librosa

def extract_formants(signal, sample_rate, order=None):
    if order is None:
        order = 2 + sample_rate // 1000  # Số lượng hệ số LPC

    window_length = int(0.025 * sample_rate)  # Độ dài cửa sổ là 25ms
    hop_length = int(0.01 * sample_rate)  # Bước nhảy là 10ms

    frames = librosa.util.frame(signal, frame_length=window_length, hop_length=hop_length)

    formant_freqs = []

    for frame in frames.T:
        frame = frame * np.hamming(len(frame))
        
        # Tính hệ số LPC
        try:
            lpc_coeffs = librosa.lpc(frame, order=order)
        except np.linalg.LinAlgError:
            continue  # Bỏ qua các khung không thể tính LPC

        roots = np.roots(lpc_coeffs)
        roots = roots[np.imag(roots) >= 0]

        angz = np.arctan2(np.imag(roots), np.real(roots))

        formant_freqs_frame = angz * (sample_rate / (2 * np.pi))
        formant_freqs.append(formant_freqs_frame)

    if len(formant_freqs) == 0:
        raise ValueError("Không có formant nào được trích xuất.")

    formant_freqs = np.concatenate(formant_freqs)
    mean_formant_freqs = np.mean(formant_freqs, axis=0)

    print(formant_freqs)
    print(mean_formant_freqs)

    # if len(mean_formant_freqs) < 3:
    #     raise ValueError("Không có đủ các formant để tính toán.")

    # return mean_formant_freqs[:3]

# Ví dụ sử dụng hàm trên một tệp âm thanh
audio_file = r'D:\data_analysis\speech_emotion_recognition\data\EnglishDataset\cleaned_data\Angry\03-01-05-01-01-01-03.wav'
signal, sample_rate = librosa.load(audio_file, sr=None)
formants = extract_formants(signal, sample_rate)
print(f"Formant F1: {formants[0]} Hz, Formant F2: {formants[1]} Hz, Formant F3: {formants[2]} Hz")


[ 474.75648544 3076.32386616 1868.76391544 ...  683.06070027  261.61080038
    0.        ]
11617.923743259638


TypeError: 'NoneType' object is not subscriptable

In [9]:
# Ví dụ sử dụng hàm trên một tệp âm thanh
audio_file = r'D:\data_analysis\speech_emotion_recognition\data\EnglishDataset\cleaned_data\Angry\03-01-05-01-01-01-03.wav'
signal, sample_rate = librosa.load(audio_file, sr=None)
formants = extract_formants(signal, sample_rate)
print(f"Formant F1: {formants[0]} Hz, Formant F2: {formants[1]} Hz, Formant F3: {formants[2]} Hz")

TypeError: lpc() missing 1 required keyword-only argument: 'order'

In [7]:
import librosa
import numpy as np
from scipy.signal import lfilter

def calculate_formants(file_path):
    # Load the audio file
    signal, sr = librosa.load(file_path)

    # Pre-emphasis
    pre_emphasized_signal = np.append(signal[0], signal[1:] - 0.97 * signal[:-1])

    # Compute the Linear Predictive Coding
    lpc_order = 2 + sr // 1000
    A = librosa.lpc(pre_emphasized_signal, order = lpc_order)
    lpc_coeffs = lfilter([0] + -1*A[1:], [1], signal)

    # Compute the roots of the LPC
    rts = np.roots(A)
    rts = [r for r in rts if np.imag(r) >= 0]

    # Compute the angles and sort them by frequency
    angz = np.arctan2(np.imag(rts), np.real(rts))
    frqs = sorted(angz * (sr / (2 * np.pi)))

    return frqs

Formants: [721.3824, 1401.1196, 2236.8738, 3239.1687, 4117.2256, 5096.966, 5802.739, 6933.5645, 8002.1777, 8683.482, 9260.525, 10236.741]


In [8]:
# Usage
formants = calculate_formants(r'D:\data_analysis\speech_emotion_recognition\data\EnglishDataset\cleaned_data\Angry\03-01-05-01-01-01-03.wav')
print('Formants:', formants)

formants = calculate_formants(r'D:\data_analysis\speech_emotion_recognition\data\EnglishDataset\cleaned_data\Happy\03-01-03-01-01-01-07.wav')
print('Formants:', formants)

Formants: [721.3824, 1401.1196, 2236.8738, 3239.1687, 4117.2256, 5096.966, 5802.739, 6933.5645, 8002.1777, 8683.482, 9260.525, 10236.741]
Formants: [679.07135, 1534.5476, 2292.1736, 3679.732, 4234.2246, 4325.078, 5888.1074, 6072.527, 7461.582, 8586.377, 9380.355, 10195.942]


In [9]:
formants = calculate_formants(r'D:\data_analysis\speech_emotion_recognition\data\EnglishDataset\cleaned_data\Angry\YAF_witch_angry.wav')
print('Formants:', formants)

Formants: [688.7086, 1428.6176, 2540.258, 3246.2712, 4149.397, 4606.274, 6000.5366, 6922.5654, 7912.6494, 8821.534, 9426.873, 10173.98]


In [10]:
formants = calculate_formants(r'D:\data_analysis\speech_emotion_recognition\data\EnglishDataset\cleaned_data\Sad\03-01-04-01-01-01-12.wav')
print('Formants:', formants)

Formants: [719.67535, 1441.8665, 1773.3247, 2716.288, 3936.2014, 5126.2817, 5596.3994, 6555.4165, 7550.396, 8533.014, 9380.397, 10208.539]


In [11]:
formants = calculate_formants(r'D:\data_analysis\speech_emotion_recognition\data\EnglishDataset\cleaned_data\Sad\1005_MTI_SAD_XX.wav')
print('Formants:', formants)

Formants: [516.06903, 1151.0686, 1947.4172, 2504.1653, 3575.7183, 4360.3867, 4878.4766, 5666.582, 6314.458, 6847.5005, 7261.4365, 7563.0835]


In [3]:
import os
import pandas as pd
import numpy as np
import librosa

class FeatureExtractor:
    def __init__(self, n_mfcc=20):
        self.n_mfcc = n_mfcc

    def compute_mfccs(self, signal, sample_rate):
        """
        Tính toán Mel-frequency cepstral coefficients (MFCCs) của tín hiệu âm thanh.
        """
        mfccs = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=self.n_mfcc)
        return np.mean(mfccs.T, axis=0)

    def compute_zcr(self, signal):
        """
        Tính toán Zero Crossing Rate (ZCR) của tín hiệu âm thanh.
        """
        zcr = librosa.feature.zero_crossing_rate(signal)[0]
        zcr_mean = np.mean(zcr)
        zcr_variance = np.var(zcr)
        zcr_max = np.max(zcr)
        zcr_min = np.min(zcr)
        zcr_median = np.median(zcr)
        return zcr_mean, zcr_variance, zcr_max, zcr_min, zcr_median

    def extract_pitch(self, signal, sample_rate):
        # Tính toán pitch sử dụng hàm estimate_tuning của Librosa
        pitch, _ = librosa.core.piptrack(y=signal, sr=sample_rate)
        
        # Lấy giá trị pitch trung bình
        mean_pitch = pitch.mean()
        
        return mean_pitch
    
    def extract_rms_features(self, y, frame_length=512, hop_length=256):
        """Trích xuất RMS energy và tính các giá trị thống kê."""
        rms = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]
        rms_mean = np.mean(rms)
        rms_variance = np.var(rms)
        rms_max = np.max(rms)
        rms_min = np.min(rms)
        rms_median = np.median(rms)
        return rms_mean, rms_variance, rms_max, rms_min, rms_median

    def calculate_formants(self, signal, sr):
        # Pre-emphasis
        pre_emphasized_signal = np.append(signal[0], signal[1:] - 0.97 * signal[:-1])

        # Compute the Linear Predictive Coding
        lpc_order = 2 + sr // 1000
        A = librosa.lpc(pre_emphasized_signal, order = lpc_order)

        # Compute the roots of the LPC
        rts = np.roots(A)
        rts = [r for r in rts if np.imag(r) >= 0]

        # Compute the angles and sort them by frequency
        angz = np.arctan2(np.imag(rts), np.real(rts))
        frqs = sorted(angz * (sr / (2 * np.pi)))

        mean_formant = np.mean(frqs)
        variance_formant = np.var(frqs)
        max_formant = np.max(frqs)
        min_formant = np.min(frqs)
        median_formant = np.median(frqs)
        return mean_formant, variance_formant, max_formant, min_formant, median_formant

    def feature_engineering_for_file(self, audio_file):
        """
        Trích xuất các đặc trưng MFCC, ZCR và Pitch từ một tệp âm thanh.
        """
        signal, sample_rate = librosa.load(audio_file, sr=None)
        if len(signal) >= 2048:
            mfcc_array = self.compute_mfccs(signal, sample_rate)
            zcr_mean, zcr_variance, zcr_max, zcr_min, zcr_median = self.compute_zcr(signal)
            pitch = self.extract_pitch(signal, sample_rate)
            rms_mean, rms_variance, rms_max, rms_min, rms_median = self.extract_rms_features(signal)
            mean_formant, variance_formant, max_formant, min_formant, median_formant = self.calculate_formants(signal, sample_rate)
            return mfcc_array, zcr_mean, zcr_variance, zcr_max, zcr_min, zcr_median, pitch, rms_mean, rms_variance, rms_max, rms_min, rms_median, mean_formant, variance_formant, max_formant, min_formant, median_formant
        return None

    def process_file(self, label, input_folder_path):
        """
        Xử lý tất cả các tệp WAV trong thư mục để trích xuất các đặc trưng.
        """
        # Tạo một DataFrame để lưu trữ đặc trưng MFCC
        mfcc_data = pd.DataFrame()

        for file_name in os.listdir(input_folder_path):
            if file_name.endswith('.wav'):
                file_path = os.path.join(input_folder_path, file_name)

                features = self.feature_engineering_for_file(file_path)
                if features is not None:
                    mfcc_array, zcr_mean, zcr_variance, zcr_max, zcr_min, zcr_median, pitch, rms_mean, rms_variance, rms_max, rms_min, rms_median, mean_formant, variance_formant, max_formant, min_formant, median_formant = features
                    mfcc_df = pd.DataFrame(mfcc_array.reshape(1, -1), columns=[f'mfcc_{i+1}' for i in range(len(mfcc_array))])

                    mfcc_df['label'] = label
                    mfcc_df['file'] = file_name
                    mfcc_df['zcr_mean'] = zcr_mean
                    mfcc_df['zcr_variance'] = zcr_variance
                    mfcc_df['zcr_max'] = zcr_max
                    mfcc_df['zcr_min'] = zcr_min
                    mfcc_df['zcr_median'] = zcr_median
                    mfcc_df['Pitch'] = pitch
                    mfcc_df['rms_mean'] = rms_mean
                    mfcc_df['rms_variance'] = rms_variance
                    mfcc_df['rms_max'] = rms_max
                    mfcc_df['rms_min'] = rms_min
                    mfcc_df['rms_median'] = rms_median
                    #mean_formant, variance_formant, max_formant, min_formant, median_formant
                    mfcc_df['mean_formant'] = mean_formant
                    mfcc_df['variance_formant'] = variance_formant
                    mfcc_df['max_formant'] = max_formant
                    mfcc_df['min_formant'] = min_formant
                    mfcc_df['median_formant'] = median_formant
                    mfcc_data = pd.concat([mfcc_data, mfcc_df], ignore_index=True)
            
        return mfcc_data

    def process_folder(self, input_folder, output_csv_path):
        """
        Xử lý tất cả các thư mục con trong thư mục đầu vào để trích xuất các đặc trưng và lưu chúng vào một tệp CSV.
        """
        # Tạo một DataFrame để lưu trữ dữ liệu MFCC
        mfcc_data = pd.DataFrame()

        # Duyệt qua tất cả các thư mục con trong thư mục đầu vào
        for root, dirs, files in os.walk(input_folder):
            for folder in dirs:
                # Xác định đường dẫn của thư mục con trong thư mục đầu vào
                subdirectory_input = os.path.join(root, folder)
                # Xử lý các tệp trong thư mục con và thêm dữ liệu MFCC vào DataFrame
                mfcc_df = self.process_file(folder, subdirectory_input)
                mfcc_data = pd.concat([mfcc_data, mfcc_df], ignore_index=True)

        # Lưu dữ liệu MFCC vào tệp CSV
        mfcc_data.to_csv(output_csv_path, index=False)

In [5]:
featureExtractor = FeatureExtractor(40)
featureExtractor.process_folder('D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/cleaned_data','feature.csv')